<a href="https://colab.research.google.com/github/norman-AI-2025/hackathon-2025/blob/main/Loan_Risk_App_merged_V7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import subprocess
import time
import os
import sys
import re
import urllib.request

# ==========================================
# PART 1: CLEANUP & SETUP
# ==========================================

print("🧹 Cleaning up old processes...")
# Force kill any lingering processes to prevent conflicts
subprocess.run(["pkill", "cloudflared"])
subprocess.run(["pkill", "streamlit"])
time.sleep(2)

print("📦 Installing dependencies...")
# Installing essential libraries for the app
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "streamlit", "pandas", "transformers", "torch"])

# ==========================================
# PART 2: CREATE APP FILE (Unified Logic)
# ==========================================

print("📝 Writing application file...")

loan_risk_app_code = '''
import pandas as pd
import streamlit as st
import numpy as np
import time
from datetime import date

# --- CONSTANTS ---
# Personal Loan Constants
PAGES_PERSONAL = {1: "Applicant Info", 2: "Guarantor Info", 3: "Loan Details", 4: "Results"}
GENDER_OPTIONS = ["Male", "Female", "Other"]
MARITAL_OPTIONS = ["Single", "Married", "Divorced", "Widowed"]
EDUCATION_OPTIONS = ["High School", "Graduate", "Post-Graduate", "PhD", "Other"]
EMPLOYMENT_OPTIONS = ["Salaried", "Self-Employed", "Business Owner", "Unemployed", "Retired"]
RELATIONSHIP_OPTIONS = ["Spouse", "Parent", "Sibling", "Friend", "Company", "Other"]
PURPOSE_OPTIONS = ["Car Loan", "Education", "Personal", "Renovation", "Business Expansion", "Other"]
COLLATERAL_OPTIONS = ["Real Estate", "Vehicle", "Fixed Deposit", "None"]

# Business Loan Constants
INDUSTRY_RISK = {
    "Healthcare/Education": 10,
    "Manufacturing/Tech Services": 8,
    "Retail/e-Commerce": 6,
    "Construction/Transport": 4,
    "Hospitality/Energy": 2
}

# --- AI MODEL SETUP (SHARED) ---
@st.cache_resource
def load_nlp_model():
    try:
        from transformers import pipeline
        # Using Zero-Shot Classification for intent analysis
        return pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-1")
    except: return None

classifier = load_nlp_model()

def compute_text_score(text):
    """
    Analyzes the intent of the text and assigns a risk score based on the topic.
    """
    if classifier is None or not text or len(str(text)) < 5: return 50.0

    # Define topics and their inherent risk (0-100)
    risk_categories = {
        "investment or education": 20,      # Low Risk (Growth)
        "home improvement": 30,             # Low Risk (Asset)
        "business expansion": 25,           # Low-Medium Risk (Business Growth)
        "debt consolidation": 50,           # Medium Risk (Management)
        "medical emergency": 65,            # Medium-High Risk (Shock)
        "luxury purchase or vacation": 85,  # High Risk (Discretionary)
        "financial distress": 95            # Very High Risk (Instability)
    }

    try:
        labels = list(risk_categories.keys())
        result = classifier(str(text), labels, multi_label=False)

        # Calculate weighted score based on probability of each topic
        final_score = 0
        scores = dict(zip(result['labels'], result['scores']))

        for label, prob in scores.items():
            final_score += prob * risk_categories[label]

        return final_score
    except: return 50.0

def compute_fusion_risk(numeric_score, text, text_multiplier=0.2):
    """
    Combines numeric score with text analysis for both Personal and Business loans.
    RISK LOGIC: High Numeric Score = High Risk. High Text Score = High Risk.
    """
    text_score = compute_text_score(text)

    # Risk Factor: If text is risky (>50), it INCREASES the score.
    # If text is safe (<50), it DECREASES the score.
    risk_factor = (text_score - 50.0) / 50.0
    multiplier = 1.0 + (risk_factor * text_multiplier)

    return multiplier, text_score

# --- PERSONAL LOAN MODELS (STANDARD) ---

def compute_personal_numeric_risk_scores(data):
    score = 0
    breakdown = {}

    # 1. Income & DTI (Stricter)
    total_income = data['app_monthly_income'] + data['app_other_monthly_income']
    total_commitments = data['app_total_monthly_loan_repayment'] + data['app_other_fixed_monthly_commitments']
    dti = total_commitments / (total_income + 1e-6)

    dti_score = 0
    if dti > 0.50: dti_score = 35
    elif dti > 0.40: dti_score = 25
    elif dti > 0.30: dti_score = 15
    score += dti_score
    breakdown['DTI Risk'] = dti_score

    # 2. Credit Utilization (Stricter)
    util = data['app_credit_card_outstanding'] / (data['app_total_credit_limit'] + 1e-6)
    util_score = 0
    if util > 0.70: util_score = 25
    elif util > 0.50: util_score = 15
    elif util > 0.30: util_score = 10
    score += util_score
    breakdown['Credit Util Risk'] = util_score

    # 3. Existing Commitments (Heavier Penalties)
    comm_score = 0
    if data['app_has_mortgage']: comm_score += 8
    if data['app_has_car_loan']: comm_score += 8
    if data['app_has_personal_loan']: comm_score += 8
    if data['app_num_credit_cards'] > 3: comm_score += 5
    comm_score = min(comm_score, 30)
    score += comm_score
    breakdown['Commitments Risk'] = comm_score

    # 4. Employment & Stability (Heavier Penalties)
    emp_score = 0
    if data['app_employment_status'] == "Unemployed": emp_score += 25
    elif data['app_employment_status'] == "Retired": emp_score += 10
    elif data['app_employment_status'] == "Self-Employed": emp_score += 5
    if data['app_years_in_job'] < 1: emp_score += 10
    elif data['app_years_in_job'] < 2: emp_score += 5
    score += emp_score
    breakdown['Employment Stability'] = emp_score

    # 5. Demographics (Minor Adjustments)
    demo_score = 0
    if data['app_age'] < 21 or data['app_age'] > 70: demo_score += 5
    if data['app_dependents'] > 4: demo_score += 5
    if data['app_education'] == "High School": demo_score += 5
    score += demo_score
    breakdown['Demographics'] = demo_score

    # 6. Security/Collateral (Higher Unsecured Penalty)
    sec_score = 0
    if not data['loan_is_secured']:
        sec_score = 25
    else:
        ltv = data['loan_amount_requested'] / (data['loan_collateral_value'] + 1e-6)
        if ltv > 0.9: sec_score = 10
    score += sec_score
    breakdown['Collateral Risk'] = sec_score

    # Penalties (Hard Adds)
    if data['app_in_legal_proceedings']:
        score += 40
        breakdown['Legal Penalty'] = 40
    if data['app_convicted_financial_crime']:
        score += 100
        breakdown['Crime Penalty'] = 100

    # Guarantor Mitigation
    if data['guar_exists']:
        guar_income_ratio = data['guar_monthly_income'] / (total_income + 1e-6)
        if guar_income_ratio > 0.5:
            score -= 15
            breakdown['Guarantor Bonus'] = -15
        elif guar_income_ratio > 0.2:
            score -= 5
            breakdown['Guarantor Bonus'] = -5

    final_score = np.clip(score, 0, 100)
    return final_score, breakdown

# --- BUSINESS LOAN MODELS (UPDATED TO MATCH RISK LOGIC) ---
# Logic: 0 = Low Risk, 100 = High Risk

def compute_business_numeric_score(data):
    score = 0
    breakdown = {}
    explanations = [] # Logs the "Why" for the UI

    # ==========================================
    # 1. REVENUE & FINANCIAL RISK (Base Risk)
    # ==========================================
    rev_risk = 0
    annual_rev = max(data['biz_rev_year_2'], data['biz_avg_monthly_revenue'] * 12)

    # Low Revenue Penalty (Adds Risk)
    if annual_rev < 50000:
        rev_risk += 20
        explanations.append("⚠️ Micro-Business Revenue (<$50k) +20 Risk")
    elif annual_rev < 200000:
        rev_risk += 10
        explanations.append("Small Scale Revenue +10 Risk")

    # Revenue Trend Risk
    if data['biz_rev_year_1'] > 0:
        growth = (data['biz_rev_year_2'] - data['biz_rev_year_1']) / data['biz_rev_year_1']
        if growth < -0.10:
            rev_risk += 15
            explanations.append(f"⚠️ Declining Revenue ({int(growth*100)}% YoY) +15 Risk")

    # Revenue Strength Bonus (Reduces Risk)
    if annual_rev > 1000000:
        rev_risk -= 10
        explanations.append("✅ Strong Revenue Scale (>$1M) -10 Risk")

    score += max(0, rev_risk)
    breakdown['Revenue Risk'] = max(0, rev_risk)

    # ==========================================
    # 2. PROFITABILITY RISK
    # ==========================================
    prof_risk = 0
    if data['biz_rev_year_2'] > 0:
        actual_margin = data['biz_profit_year_2'] / data['biz_rev_year_2']
        if actual_margin < 0:
            prof_risk += 25
            explanations.append("⚠️ Loss Making Business +25 Risk")
        elif actual_margin < 0.05:
            prof_risk += 10
            explanations.append("Thin Margins (<5%) +10 Risk")
        elif actual_margin > 0.20:
            prof_risk -= 5
            explanations.append("✅ Strong Profit Margins -5 Risk")

    score += prof_risk
    breakdown['Profitability Risk'] = prof_risk

    # ==========================================
    # 3. LIQUIDITY & DEBT SERVICE RISK (DSCR)
    # ==========================================
    liq_risk = 0
    monthly_free_cash = data['biz_avg_monthly_revenue'] - data['biz_avg_monthly_expenses'] - data['biz_existing_loan_monthly_repayment']
    est_new_repayment = data['loan_amount_requested'] / (data['loan_tenure_months'] + 1e-6) * 1.1

    if est_new_repayment > 0:
        dscr = monthly_free_cash / est_new_repayment
        if dscr < 1.0:
            liq_risk += 30
            explanations.append(f"⚠️ Negative Cashflow Coverage ({dscr:.1f}x) +30 Risk")
        elif dscr < 1.25:
            liq_risk += 15
            explanations.append(f"Tight Cashflow ({dscr:.1f}x) +15 Risk")
        elif dscr > 2.0:
            liq_risk -= 10
            explanations.append(f"✅ Strong DSCR ({dscr:.1f}x) -10 Risk")

    # Burn Rate / Runway
    monthly_burn = data['biz_avg_monthly_expenses'] + data['biz_existing_loan_monthly_repayment']
    if monthly_burn > 0:
        months_runway = data['biz_cash_at_bank'] / monthly_burn
        if months_runway < 1.0:
            liq_risk += 10
            explanations.append("⚠️ Low Cash Reserves (<1 Month) +10 Risk")
        elif months_runway > 4.0:
            liq_risk -= 5
            explanations.append("✅ Healthy Cash Reserves -5 Risk")

    score += max(0, liq_risk)
    breakdown['Liquidity Risk'] = max(0, liq_risk)

    # ==========================================
    # 4. CREDIT & STABILITY
    # ==========================================
    stab_risk = 0

    # Credit Score Risk
    if data['biz_credit_score'] < 600:
        stab_risk += 25
        explanations.append("⚠️ Poor Credit Score (<600) +25 Risk")
    elif data['biz_credit_score'] < 700:
        stab_risk += 10
        explanations.append("Average Credit Score +10 Risk")
    elif data['biz_credit_score'] > 780:
        stab_risk -= 10
        explanations.append("✅ Excellent Credit Score -10 Risk")

    # Business Age
    if data['biz_years_in_operation'] < 2:
        stab_risk += 15
        explanations.append("Startup Phase (<2 Years) +15 Risk")

    # Collateral Mitigation (Reduces Risk)
    if data['loan_is_secured']:
        ltv = data['loan_amount_requested'] / (data['loan_collateral_value'] + 1e-6)
        if ltv < 0.70:
            stab_risk -= 20
            explanations.append("✅ Fully Secured (LTV <70%) -20 Risk")
        else:
            stab_risk -= 10
            explanations.append("✅ Partially Secured -10 Risk")
    else:
        stab_risk += 10
        explanations.append("Unsecured Loan +10 Risk")

    score += stab_risk
    breakdown['Stability Risk'] = stab_risk

    # ==========================================
    # 5. HARD PENALTIES (Immediate Adds)
    # ==========================================
    penalty = 0
    if data['biz_tax_arrears'] or data['biz_epf_arrears']:
        penalty += 40
        explanations.append("❌ Statutory Arrears Detected +40 Risk")

    if data['biz_legal_dispute']:
        penalty += 30
        explanations.append("❌ Active Legal Dispute +30 Risk")

    if data['biz_blacklist_history']:
        penalty += 100
        explanations.append("❌ BLACKLISTED +100 Risk")

    score += penalty
    breakdown['Penalties'] = penalty

    # Cap final score
    return np.clip(score, 0, 100), breakdown, explanations

# --- STATE INIT ---
def init_state():
    if 'loan_type' not in st.session_state: st.session_state.loan_type = None
    if 'current_page' not in st.session_state: st.session_state.current_page = 1

    # Define all default values
    defaults = {
        # --- PERSONAL LOAN DATA ---
        'app_age': 30, 'app_gender': 'Male', 'app_marital_status': 'Single',
        'app_dependents': 0, 'app_education': 'Graduate', 'app_employment_status': 'Salaried',
        'app_years_in_job': 2, 'app_total_work_experience': 5,
        'app_monthly_income': 5000, 'app_other_monthly_income': 0,
        'app_has_mortgage': False, 'app_has_car_loan': False, 'app_has_personal_loan': False,
        'app_has_credit_card': True, 'app_num_credit_cards': 1,
        'app_total_credit_limit': 10000, 'app_credit_card_outstanding': 1000,
        'app_total_monthly_loan_repayment': 500, 'app_other_fixed_monthly_commitments': 200,
        'app_in_legal_proceedings': False, 'app_convicted_financial_crime': False,

        'guar_exists': False, 'guar_relationship': 'Spouse', 'guar_age': 30,
        'guar_employment_status': 'Salaried', 'guar_monthly_income': 0, 'guar_other_monthly_income': 0,
        'guar_total_monthly_loan_repayment': 0,
        'guar_has_credit_card': False, 'guar_num_credit_cards': 0,
        'guar_total_credit_limit': 0, 'guar_credit_card_outstanding': 0,

        'loan_amount_requested': 20000, 'loan_tenure_months': 24, 'loan_purpose': 'Personal',
        'loan_is_secured': False, 'loan_collateral_type': 'None', 'loan_collateral_value': 0,
        'loan_application_date': date.today(),
        'loan_essay_text': "I plan to use this loan for...",

        # --- BUSINESS LOAN DATA ---
        # Page 1 - Profile
        'biz_name': "", 'biz_reg_no': "", 'biz_entity_type': "Sole Proprietorship",
        'biz_year_established': 2020, 'biz_industry': list(INDUSTRY_RISK.keys())[2],
        'biz_years_in_operation': 3, 'biz_num_owners': 1, 'biz_main_product': "",
        'biz_customer_concentration_high': False, 'biz_num_employees': 5, 'biz_premise_ownership': "Rented",

        # Page 2 - Financials
        'biz_rev_year_1': 0, 'biz_profit_year_1': 0,
        'biz_rev_year_2': 0, 'biz_profit_year_2': 0,
        'biz_avg_monthly_revenue': 0, 'biz_avg_monthly_expenses': 0,
        'biz_cash_at_bank': 0, 'biz_existing_loan_count': 0,
        'biz_existing_loan_outstanding': 0, 'biz_existing_loan_monthly_repayment': 0,
        'biz_has_overdraft': False, 'biz_overdraft_limit': 0, 'biz_overdraft_utilization': 0,
        'biz_tax_arrears': False, 'biz_epf_arrears': False,
        'biz_credit_score': 700,

        # Page 3 - Loan & Guarantee
        'has_personal_guarantee': False, 'guar_owner_income': 0, 'guar_owner_existing_debt': 0,
        'biz_legal_dispute': False, 'biz_blacklist_history': False, 'biz_high_risk_sector': False,
        'biz_essay_text': "We need this loan to expand our inventory...",

        # --- RESULTS ---
        'numeric_score': 0, 'text_score': 0, 'risk_score': 0, 'score_multiplier': 1.0,
        'risk_category': 'N/A', 'risk_breakdown': {}, 'risk_explanations': []
    }

    if 'form_data' not in st.session_state:
        st.session_state.form_data = defaults
    else:
        # Fix for KeyError: Ensure all new keys exist in current session
        for key, value in defaults.items():
            if key not in st.session_state.form_data:
                st.session_state.form_data[key] = value

# --- PAGE RENDERERS ---

def page_loan_type_selection():
    st.header("Start Application")
    st.info("Please select the type of loan you are applying for.")

    col1, col2 = st.columns(2)
    if col1.button("👤 Personal Loan", use_container_width=True):
        st.session_state.loan_type = "Personal"
        st.rerun()

    if col2.button("🏢 Business Loan", use_container_width=True):
        st.session_state.loan_type = "Business"
        st.rerun()

# === PERSONAL LOAN PAGES ===
def personal_page_1():
    st.header("Personal Applicant Info")
    c1, c2, c3 = st.columns(3)
    st.session_state.form_data['app_age'] = c1.number_input("Age", 18, 100, st.session_state.form_data['app_age'])
    st.session_state.form_data['app_gender'] = c2.selectbox("Gender", GENDER_OPTIONS, index=GENDER_OPTIONS.index(st.session_state.form_data['app_gender']))
    st.session_state.form_data['app_marital_status'] = c3.selectbox("Marital Status", MARITAL_OPTIONS, index=MARITAL_OPTIONS.index(st.session_state.form_data['app_marital_status']))

    c4, c5, c6 = st.columns(3)
    st.session_state.form_data['app_dependents'] = c4.number_input("Dependents", 0, 20, st.session_state.form_data['app_dependents'])
    st.session_state.form_data['app_education'] = c5.selectbox("Education", EDUCATION_OPTIONS, index=EDUCATION_OPTIONS.index(st.session_state.form_data['app_education']))
    st.session_state.form_data['app_employment_status'] = c6.selectbox("Employment Status", EMPLOYMENT_OPTIONS, index=EMPLOYMENT_OPTIONS.index(st.session_state.form_data['app_employment_status']))

    c7, c8 = st.columns(2)
    st.session_state.form_data['app_years_in_job'] = c7.number_input("Years in Current Job", 0, 60, st.session_state.form_data['app_years_in_job'])
    st.session_state.form_data['app_total_work_experience'] = c8.number_input("Total Work Experience", 0, 60, st.session_state.form_data['app_total_work_experience'])

    st.markdown("---")
    st.subheader("Financials")
    f1, f2 = st.columns(2)
    st.session_state.form_data['app_monthly_income'] = f1.number_input("Monthly Income ($)", 0, value=st.session_state.form_data['app_monthly_income'], step=100)
    st.session_state.form_data['app_other_monthly_income'] = f2.number_input("Other Monthly Income ($)", 0, value=st.session_state.form_data['app_other_monthly_income'], step=100)

    st.subheader("Commitments")
    chk1, chk2, chk3, chk4 = st.columns(4)
    st.session_state.form_data['app_has_mortgage'] = chk1.checkbox("Mortgage?", st.session_state.form_data['app_has_mortgage'])
    st.session_state.form_data['app_has_car_loan'] = chk2.checkbox("Car Loan?", st.session_state.form_data['app_has_car_loan'])
    st.session_state.form_data['app_has_personal_loan'] = chk3.checkbox("Personal Loan?", st.session_state.form_data['app_has_personal_loan'])
    st.session_state.form_data['app_has_credit_card'] = chk4.checkbox("Credit Card?", st.session_state.form_data['app_has_credit_card'])

    cc1, cc2, cc3 = st.columns(3)
    st.session_state.form_data['app_num_credit_cards'] = cc1.number_input("Num Credit Cards", 0, value=st.session_state.form_data['app_num_credit_cards'])
    st.session_state.form_data['app_total_credit_limit'] = cc2.number_input("Total Credit Limit", 0, value=st.session_state.form_data['app_total_credit_limit'], step=500)
    st.session_state.form_data['app_credit_card_outstanding'] = cc3.number_input("Credit Card Outstanding", 0, value=st.session_state.form_data['app_credit_card_outstanding'], step=100)

    rep1, rep2 = st.columns(2)
    st.session_state.form_data['app_total_monthly_loan_repayment'] = rep1.number_input("Total Monthly Loan Repayments", 0, value=st.session_state.form_data['app_total_monthly_loan_repayment'], step=100)
    st.session_state.form_data['app_other_fixed_monthly_commitments'] = rep2.number_input("Other Fixed Commitments", 0, value=st.session_state.form_data['app_other_fixed_monthly_commitments'], step=100)

    st.markdown("---")
    st.subheader("Legal")
    st.session_state.form_data['app_in_legal_proceedings'] = st.checkbox("Active Legal Proceedings?", st.session_state.form_data['app_in_legal_proceedings'])
    st.session_state.form_data['app_convicted_financial_crime'] = st.checkbox("Convicted of Financial Crime?", st.session_state.form_data['app_convicted_financial_crime'])

def personal_page_2():
    st.header("Personal Guarantor Info")
    st.session_state.form_data['guar_exists'] = st.radio("Do you have a guarantor?", [True, False], index=0 if st.session_state.form_data['guar_exists'] else 1)

    if st.session_state.form_data['guar_exists']:
        if st.session_state.form_data['guar_relationship'] not in RELATIONSHIP_OPTIONS:
            st.session_state.form_data['guar_relationship'] = RELATIONSHIP_OPTIONS[0]
        if st.session_state.form_data['guar_age'] < 18: st.session_state.form_data['guar_age'] = 18

        g1, g2, g3 = st.columns(3)
        st.session_state.form_data['guar_relationship'] = g1.selectbox("Relationship", RELATIONSHIP_OPTIONS, index=RELATIONSHIP_OPTIONS.index(st.session_state.form_data['guar_relationship']))
        st.session_state.form_data['guar_age'] = g2.number_input("Guarantor Age", 18, 100, st.session_state.form_data['guar_age'])
        st.session_state.form_data['guar_employment_status'] = g3.selectbox("Guarantor Employment", EMPLOYMENT_OPTIONS, index=EMPLOYMENT_OPTIONS.index(st.session_state.form_data['guar_employment_status']))

        g4, g5, g6 = st.columns(3)
        st.session_state.form_data['guar_monthly_income'] = g4.number_input("Guarantor Income", 0, value=st.session_state.form_data['guar_monthly_income'], step=100)
        st.session_state.form_data['guar_other_monthly_income'] = g5.number_input("Guarantor Other Income", 0, value=st.session_state.form_data['guar_other_monthly_income'], step=100)
        st.session_state.form_data['guar_total_monthly_loan_repayment'] = g6.number_input("Guarantor Repayments", 0, value=st.session_state.form_data['guar_total_monthly_loan_repayment'], step=100)

        st.markdown("---")
        st.subheader("Guarantor Credit")
        st.session_state.form_data['guar_has_credit_card'] = st.checkbox("Guarantor Has Credit Card?", st.session_state.form_data['guar_has_credit_card'])

        gc1, gc2, gc3 = st.columns(3)
        st.session_state.form_data['guar_num_credit_cards'] = gc1.number_input("Guarantor Num Cards", 0, value=st.session_state.form_data['guar_num_credit_cards'])
        st.session_state.form_data['guar_total_credit_limit'] = gc2.number_input("Guarantor Credit Limit", 0, value=st.session_state.form_data['guar_total_credit_limit'], step=500)
        st.session_state.form_data['guar_credit_card_outstanding'] = gc3.number_input("Guarantor Outstanding", 0, value=st.session_state.form_data['guar_credit_card_outstanding'], step=100)
    else:
        st.session_state.form_data['guar_monthly_income'] = 0
        st.session_state.form_data['guar_age'] = 0

def personal_page_3():
    st.header("Personal Loan Details")
    l1, l2 = st.columns(2)
    st.session_state.form_data['loan_amount_requested'] = l1.number_input("Amount Requested ($)", 1000, value=st.session_state.form_data['loan_amount_requested'], step=1000)
    st.session_state.form_data['loan_tenure_months'] = l2.number_input("Tenure (Months)", 6, 360, st.session_state.form_data['loan_tenure_months'])

    l3, l4 = st.columns(2)
    st.session_state.form_data['loan_purpose'] = l3.selectbox("Purpose", PURPOSE_OPTIONS, index=PURPOSE_OPTIONS.index(st.session_state.form_data['loan_purpose']))
    st.session_state.form_data['loan_application_date'] = l4.date_input("Application Date", value=st.session_state.form_data['loan_application_date'])

    st.markdown("---")
    st.session_state.form_data['loan_is_secured'] = st.radio("Is this a Secured Loan?", [True, False], index=0 if st.session_state.form_data['loan_is_secured'] else 1)

    if st.session_state.form_data['loan_is_secured']:
        if st.session_state.form_data['loan_collateral_type'] not in COLLATERAL_OPTIONS:
             st.session_state.form_data['loan_collateral_type'] = COLLATERAL_OPTIONS[0]

        c1, c2 = st.columns(2)
        st.session_state.form_data['loan_collateral_type'] = c1.selectbox("Collateral Type", COLLATERAL_OPTIONS, index=COLLATERAL_OPTIONS.index(st.session_state.form_data['loan_collateral_type']))
        st.session_state.form_data['loan_collateral_value'] = c2.number_input("Collateral Value ($)", 0, value=st.session_state.form_data['loan_collateral_value'], step=1000)
    else:
        st.session_state.form_data['loan_collateral_type'] = 'None'
        st.session_state.form_data['loan_collateral_value'] = 0

    st.subheader("Personal Essay")
    st.caption("Please describe why you need this loan. The AI will analyze your intent.")
    st.session_state.form_data['loan_essay_text'] = st.text_area("Reason for loan:", value=st.session_state.form_data['loan_essay_text'], height=150)

def personal_page_4(mult):
    st.header("Personal Risk Results")
    if st.button("Calculate Risk", type="primary"):
        with st.spinner("Analyzing Personal Risk Profile..."):
            numeric_score, breakdown = compute_personal_numeric_risk_scores(st.session_state.form_data)
            st.session_state.form_data['numeric_score'] = numeric_score
            st.session_state.form_data['risk_breakdown'] = breakdown

            multiplier, text_score = compute_fusion_risk(numeric_score, st.session_state.form_data['loan_essay_text'], mult)

            # For Personal: High Score = High Risk.
            final_risk = numeric_score * multiplier
            st.session_state.form_data['risk_score'] = np.clip(final_risk, 0, 100)
            st.session_state.form_data['score_multiplier'] = multiplier
            st.session_state.form_data['text_score'] = text_score

            if final_risk >= 75: st.session_state.form_data['risk_category'] = "High Risk"
            elif final_risk >= 35: st.session_state.form_data['risk_category'] = "Medium Risk"
            else: st.session_state.form_data['risk_category'] = "Low Risk"

            time.sleep(1)
        st.success("Assessment Complete")

        r1, r2, r3, r4 = st.columns(4)
        r1.metric("Risk Category", st.session_state.form_data['risk_category'])
        r2.metric("Risk Score (Lower is Better)", f"{st.session_state.form_data['risk_score']:.1f}")
        r3.metric("Numeric Base", f"{st.session_state.form_data['numeric_score']:.1f}")
        r4.metric("Intent Impact", f"x{st.session_state.form_data['score_multiplier']:.2f}")

        st.markdown("### Score Breakdown")
        df = pd.DataFrame([
            {"Component": k, "Points": v}
            for k, v in st.session_state.form_data['risk_breakdown'].items()
        ])
        st.dataframe(df, use_container_width=True)
        st.info(f"Text Analysis Score: {st.session_state.form_data['text_score']:.1f}/100 (Higher = Riskier Topic)")

# === BUSINESS LOAN PAGES ===

def business_page_1():
    st.header("Page 1: Business Profile")

    c1, c2 = st.columns(2)
    st.session_state.form_data['biz_name'] = c1.text_input("Business Name", value=st.session_state.form_data['biz_name'])
    st.session_state.form_data['biz_reg_no'] = c2.text_input("Registration No", value=st.session_state.form_data['biz_reg_no'])

    c3, c4 = st.columns(2)
    st.session_state.form_data['biz_entity_type'] = c3.selectbox("Entity Type", ["Sole Proprietorship", "Partnership", "LLP", "Private Ltd", "Public Ltd"], index=0)
    st.session_state.form_data['biz_year_established'] = c4.number_input("Year Established", 1900, 2025, value=st.session_state.form_data['biz_year_established'])

    if st.session_state.form_data['biz_industry'] not in INDUSTRY_RISK:
        st.session_state.form_data['biz_industry'] = list(INDUSTRY_RISK.keys())[0]
    st.session_state.form_data['biz_industry'] = st.selectbox("Industry", list(INDUSTRY_RISK.keys()), index=list(INDUSTRY_RISK.keys()).index(st.session_state.form_data['biz_industry']))

    c5, c6 = st.columns(2)
    st.session_state.form_data['biz_years_in_operation'] = c5.number_input("Years in Operation", 0, 100, value=st.session_state.form_data['biz_years_in_operation'])
    st.session_state.form_data['biz_num_owners'] = c6.number_input("Num Owners", 1, 20, value=st.session_state.form_data['biz_num_owners'])

    st.session_state.form_data['biz_main_product'] = st.text_input("Main Product/Service", value=st.session_state.form_data['biz_main_product'])

    c7, c8, c9 = st.columns(3)
    st.session_state.form_data['biz_customer_concentration_high'] = c7.checkbox("High Customer Concentration?", value=st.session_state.form_data['biz_customer_concentration_high'])
    st.session_state.form_data['biz_num_employees'] = c8.number_input("Num Employees", 0, 5000, value=st.session_state.form_data['biz_num_employees'])
    st.session_state.form_data['biz_premise_ownership'] = c9.selectbox("Premise Ownership", ["Rented", "Owned", "Leased"], index=0)

def business_page_2():
    st.header("Page 2: Financials & Debts")

    st.subheader("Performance (Last 2 Years)")
    y1_col, y2_col = st.columns(2)
    with y1_col:
        st.session_state.form_data['biz_rev_year_1'] = st.number_input("Revenue Year 1 ($)", 0, value=st.session_state.form_data['biz_rev_year_1'], step=1000)
        st.session_state.form_data['biz_profit_year_1'] = st.number_input("Profit Year 1 ($)", value=st.session_state.form_data['biz_profit_year_1'], step=1000)
    with y2_col:
        st.session_state.form_data['biz_rev_year_2'] = st.number_input("Revenue Year 2 ($)", 0, value=st.session_state.form_data['biz_rev_year_2'], step=1000)
        st.session_state.form_data['biz_profit_year_2'] = st.number_input("Profit Year 2 ($)", value=st.session_state.form_data['biz_profit_year_2'], step=1000)

    st.subheader("Current Cashflow")
    c1, c2, c3 = st.columns(3)
    st.session_state.form_data['biz_avg_monthly_revenue'] = c1.number_input("Avg Monthly Revenue", 0, value=st.session_state.form_data['biz_avg_monthly_revenue'], step=1000)
    st.session_state.form_data['biz_avg_monthly_expenses'] = c2.number_input("Avg Monthly Expenses", 0, value=st.session_state.form_data['biz_avg_monthly_expenses'], step=1000)
    st.session_state.form_data['biz_cash_at_bank'] = c3.number_input("Cash at Bank", 0, value=st.session_state.form_data['biz_cash_at_bank'], step=1000)

    st.subheader("Existing Debts / Credit")
    c_score_col, _ = st.columns(2)
    st.session_state.form_data['biz_credit_score'] = c_score_col.number_input("Business/Director Credit Score", 300, 900, value=st.session_state.form_data['biz_credit_score'])

    d1, d2, d3 = st.columns(3)
    st.session_state.form_data['biz_existing_loan_count'] = d1.number_input("Existing Loan Count", 0, value=st.session_state.form_data['biz_existing_loan_count'])
    st.session_state.form_data['biz_existing_loan_outstanding'] = d2.number_input("Total Outstanding", 0, value=st.session_state.form_data['biz_existing_loan_outstanding'], step=1000)
    st.session_state.form_data['biz_existing_loan_monthly_repayment'] = d3.number_input("Total Monthly Repayment", 0, value=st.session_state.form_data['biz_existing_loan_monthly_repayment'], step=500)

    st.markdown("---")
    st.session_state.form_data['biz_has_overdraft'] = st.checkbox("Has Overdraft?", value=st.session_state.form_data['biz_has_overdraft'])
    if st.session_state.form_data['biz_has_overdraft']:
        o1, o2 = st.columns(2)
        st.session_state.form_data['biz_overdraft_limit'] = o1.number_input("OD Limit", 0, value=st.session_state.form_data['biz_overdraft_limit'], step=1000)
        st.session_state.form_data['biz_overdraft_utilization'] = o2.number_input("OD Utilization", 0, value=st.session_state.form_data['biz_overdraft_utilization'], step=1000)

    st.markdown("---")
    st.subheader("Statutory Checks")
    c_tax, c_epf = st.columns(2)
    st.session_state.form_data['biz_tax_arrears'] = c_tax.checkbox("Tax Arrears?", value=st.session_state.form_data['biz_tax_arrears'])
    st.session_state.form_data['biz_epf_arrears'] = c_epf.checkbox("EPF Arrears?", value=st.session_state.form_data['biz_epf_arrears'])

def business_page_3():
    st.header("Page 3: Loan & Guarantees")

    st.subheader("Loan Request")
    l1, l2, l3 = st.columns(3)
    st.session_state.form_data['loan_amount_requested'] = l1.number_input("Amount ($)", 1000, value=st.session_state.form_data['loan_amount_requested'], step=5000)
    st.session_state.form_data['loan_tenure_months'] = l2.number_input("Tenure (Months)", 6, 60, value=st.session_state.form_data['loan_tenure_months'])
    st.session_state.form_data['loan_purpose'] = l3.selectbox("Purpose", PURPOSE_OPTIONS, index=PURPOSE_OPTIONS.index(st.session_state.form_data['loan_purpose']))

    st.session_state.form_data['loan_is_secured'] = st.checkbox("Is Secured Loan?", value=st.session_state.form_data['loan_is_secured'])
    if st.session_state.form_data['loan_is_secured']:
        if st.session_state.form_data['loan_collateral_type'] not in COLLATERAL_OPTIONS:
             st.session_state.form_data['loan_collateral_type'] = COLLATERAL_OPTIONS[0]
        s1, s2 = st.columns(2)
        st.session_state.form_data['loan_collateral_type'] = s1.selectbox("Collateral Type", COLLATERAL_OPTIONS, index=COLLATERAL_OPTIONS.index(st.session_state.form_data['loan_collateral_type']))
        st.session_state.form_data['loan_collateral_value'] = s2.number_input("Collateral Value", 0, value=st.session_state.form_data['loan_collateral_value'], step=5000)

    st.markdown("---")
    st.subheader("Guarantees & Risk")
    st.session_state.form_data['has_personal_guarantee'] = st.checkbox("Personal Guarantee Available?", value=st.session_state.form_data['has_personal_guarantee'])

    if st.session_state.form_data['has_personal_guarantee']:
        g1, g2 = st.columns(2)
        st.session_state.form_data['guar_owner_income'] = g1.number_input("Owner Monthly Income", 0, value=st.session_state.form_data['guar_owner_income'], step=1000)
        st.session_state.form_data['guar_owner_existing_debt'] = g2.number_input("Owner Total Debt", 0, value=st.session_state.form_data['guar_owner_existing_debt'], step=1000)

    st.markdown("---")
    c_leg, c_black, c_risk = st.columns(3)
    st.session_state.form_data['biz_legal_dispute'] = c_leg.checkbox("Legal Dispute?", value=st.session_state.form_data['biz_legal_dispute'])
    st.session_state.form_data['biz_blacklist_history'] = c_black.checkbox("Blacklist History?", value=st.session_state.form_data['biz_blacklist_history'])
    st.session_state.form_data['biz_high_risk_sector'] = c_risk.checkbox("High Risk Sector?", value=st.session_state.form_data['biz_high_risk_sector'])

    st.subheader("Essay")
    st.session_state.form_data['biz_essay_text'] = st.text_area("Business Plan / Explanation:", value=st.session_state.form_data['biz_essay_text'], height=150)

def business_page_results(mult):
    st.header("Business Risk Results")
    if st.button("Calculate Business Risk", type="primary"):
        with st.spinner("Analyzing Business Risk Profile..."):
            numeric_score, breakdown, explanations = compute_business_numeric_score(st.session_state.form_data)
            st.session_state.form_data['numeric_score'] = numeric_score
            st.session_state.form_data['risk_breakdown'] = breakdown
            st.session_state.form_data['risk_explanations'] = explanations

            # --- UNIFIED LOGIC: Higher Score = Higher Risk ---
            # We use the same fusion logic as Personal Loans now.
            multiplier, text_score = compute_fusion_risk(numeric_score, st.session_state.form_data['biz_essay_text'], mult)

            final_risk = numeric_score * multiplier
            st.session_state.form_data['risk_score'] = np.clip(final_risk, 0, 100)
            st.session_state.form_data['score_multiplier'] = multiplier
            st.session_state.form_data['text_score'] = text_score

            # Define Risk Categories (Same as Personal)
            if final_risk >= 75:
                st.session_state.form_data['risk_category'] = "High Risk (Denied)"
            elif final_risk >= 35:
                st.session_state.form_data['risk_category'] = "Medium Risk (Review)"
            else:
                st.session_state.form_data['risk_category'] = "Low Risk (Approved)"

            time.sleep(1)

        st.success("Assessment Complete")

        # Color Logic: Low Score (Green) is now Good
        color = "green" if st.session_state.form_data['risk_score'] < 35 else "orange" if st.session_state.form_data['risk_score'] < 75 else "red"

        st.markdown(f"### Assessment: :{color}[{st.session_state.form_data['risk_category']}]")

        r1, r2, r3, r4 = st.columns(4)
        r1.metric("Total Risk Score (Lower is Better)", f"{st.session_state.form_data['risk_score']:.1f}")
        r2.metric("Numeric Base Risk", f"{st.session_state.form_data['numeric_score']:.1f}")
        r3.metric("AI Intent Impact", f"x{st.session_state.form_data['score_multiplier']:.2f}")
        r4.metric("Text Analysis Risk", f"{st.session_state.form_data['text_score']:.1f}/100")

        st.markdown("---")
        st.subheader("📝 Detailed Risk Breakdown")

        # 1. Show the Table
        df = pd.DataFrame([
            {"Component": k, "Risk Points": f"{v:.1f}"}
            for k, v in st.session_state.form_data['risk_breakdown'].items()
        ])
        st.dataframe(df, use_container_width=True)

        # 2. Show the Explanations
        with st.expander("🔎 Click to see Risk Factors"):
            if st.session_state.form_data.get('risk_explanations'):
                for exp in st.session_state.form_data['risk_explanations']:
                    # Highlight risks in Red, mitigations in Green
                    if "+" in exp: # Positive points mean RISK added
                        st.markdown(f":red[**{exp}**]")
                    elif "-" in exp: # Negative points mean RISK reduced
                        st.markdown(f":green[**{exp}**]")
                    else:
                        st.write(exp)
            else:
                st.write("No specific risk drivers identified.")

def main():
    init_state()
    st.set_page_config(page_title="Unified Loan App", layout="wide")

    # Sidebar Navigation
    with st.sidebar:
        st.title("Navigation")
        if st.button("🏠 Home / Reset"):
            st.session_state.loan_type = None
            st.session_state.current_page = 1
            st.rerun()

        if st.session_state.loan_type == "Personal":
            if st.session_state.current_page > 1:
                if st.button("Previous"): st.session_state.current_page -= 1
            if st.session_state.current_page < 4:
                if st.button("Next"): st.session_state.current_page += 1
            st.info(f"Personal Loan: Page {st.session_state.current_page}/4")

        elif st.session_state.loan_type == "Business":
            if st.session_state.current_page > 1:
                if st.button("Previous"): st.session_state.current_page -= 1
            if st.session_state.current_page < 4:
                if st.button("Next"): st.session_state.current_page += 1
            st.info(f"Business Loan: Page {st.session_state.current_page}/4")

        st.markdown("---")
        mult = st.slider("AI Sensitivity", 0.0, 0.5, 0.2, 0.05)

    # Main Routing
    if st.session_state.loan_type is None:
        page_loan_type_selection()

    elif st.session_state.loan_type == "Personal":
        if st.session_state.current_page == 1: personal_page_1()
        elif st.session_state.current_page == 2: personal_page_2()
        elif st.session_state.current_page == 3: personal_page_3()
        elif st.session_state.current_page == 4: personal_page_4(mult)

    elif st.session_state.loan_type == "Business":
        if st.session_state.current_page == 1: business_page_1()
        elif st.session_state.current_page == 2: business_page_2()
        elif st.session_state.current_page == 3: business_page_3()
        elif st.session_state.current_page == 4: business_page_results(mult)

if __name__ == "__main__": main()
'''

with open('app.py', 'w') as f: f.write(loan_risk_app_code)
print("✅ Application file saved.")

# ==========================================
# PART 3: LAUNCH SYSTEM (Robust & Blocking)
# ==========================================

PORT = 8501

# 1. Launch Streamlit in Background
print(f"\\n🚀 Launching Streamlit on port {PORT}...")
# We redirect output to a file so we can debug crashes
st_log = open("streamlit.log", "w")
app_process = subprocess.Popen(
    f"streamlit run app.py --server.port {PORT} --server.address 0.0.0.0 --server.headless true",
    shell=True,
    preexec_fn=os.setsid,
    stdout=st_log,
    stderr=st_log
)

# 2. Wait for App to be Healthy
print("⏳ Waiting for app to initialize (this prevents 'Site Cant Be Reached')...")
print("   (This might take 1 minute to download AI models)")
ready = False
for i in range(60): # Wait up to 60 seconds
    try:
        with urllib.request.urlopen(f"http://localhost:{PORT}/_stcore/health", timeout=1) as response:
            if response.status == 200:
                print("\\n✅ Streamlit is ready!")
                ready = True
                break
    except:
        time.sleep(1)
        if i % 5 == 0: print(".", end="", flush=True)

if not ready:
    print("\\n❌ Error: App failed to start. Checking logs...")
    if os.path.exists("streamlit.log"):
        with open("streamlit.log", "r") as f:
            print(f.read()[-2000:]) # Print last 2000 characters
    else:
        print("No log file found.")
else:
    # 3. Install Cloudflare
    if not os.path.exists("cloudflared-linux-amd64.deb"):
        print("⬇️ Downloading Cloudflare Tunnel...")
        subprocess.run("wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb", shell=True)
        subprocess.run("dpkg -i cloudflared-linux-amd64.deb > /dev/null 2>&1", shell=True)

    # 4. Launch Tunnel (Blocking Mode Simulator)
    print("🚇 Starting Tunnel...")
    log_file = "cf.log"
    log = open(log_file, "w")

    # Start the tunnel process
    tunnel_proc = subprocess.Popen(
        ["cloudflared", "tunnel", "--url", f"http://localhost:{PORT}"],
        stdout=log,
        stderr=log,
        preexec_fn=os.setsid
    )

    # 5. Extract URL and KEEP ALIVE
    print("🔗 Generating link...")
    time.sleep(5)

    found_url = False

    # Monitoring Loop (Keeps cell running)
    try:
        while True:
            # Check if processes are alive
            if app_process.poll() is not None:
                print("\\n❌ Streamlit app died! Check streamlit.log")
                break
            if tunnel_proc.poll() is not None:
                print("\\n❌ Cloudflare Tunnel died! Check cf.log")
                break

            # Try to find URL if not found yet
            if not found_url and os.path.exists(log_file):
                with open(log_file, "r") as f:
                    txt = f.read()
                    # Regex for trycloudflare.com
                    match = re.search(r'(https:\/\/[a-zA-Z0-9-]+\.trycloudflare\.com)', txt)
                    if match:
                        url = match.group(1)
                        print(f"\\n🎯 \\033[1;32mYOUR APP IS LIVE:\\033[0m {url}")
                        print("⚠️ Keep this cell running to maintain the connection!")
                        found_url = True

            time.sleep(2)

    except KeyboardInterrupt:
        print("\\n🛑 Stopping processes...")
        tunnel_proc.terminate()
        app_process.terminate()

🧹 Cleaning up old processes...
📦 Installing dependencies...
📝 Writing application file...
✅ Application file saved.
\n🚀 Launching Streamlit on port 8501...
⏳ Waiting for app to initialize (this prevents 'Site Cant Be Reached')...
   (This might take 1 minute to download AI models)
.\n✅ Streamlit is ready!
⬇️ Downloading Cloudflare Tunnel...
🚇 Starting Tunnel...
🔗 Generating link...
\n🎯 \033YOUR APP IS LIVE:\033 https://statutory-weight-headset-switched.trycloudflare.com
⚠️ Keep this cell running to maintain the connection!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')